In [4]:
import pandas as pd
import numpy as np
from pathlib import Path
import uuid
import folium
from folium import FeatureGroup, LayerControl
from folium.plugins import HeatMap, MarkerCluster, MiniMap, Fullscreen, MeasureControl

# Loading and cleaning the dataset
cleaned_path = Path("C:/Users/rohit/OneDrive/Desktop/Uni/Dissertation/cleaned_dataset/cleaned_city_of_london_crime_for_spatial_analysis.csv")
df_city = pd.read_csv(cleaned_path)

# infer column names exactly like before
df_city.columns = [c.strip() for c in df_city.columns]
lower_map = {c.lower(): c for c in df_city.columns}

lon_col = lower_map.get("longitude")
lat_col = lower_map.get("latitude")
loc_col = lower_map.get("location")
month_col = lower_map.get("month")
crime_id_col = lower_map.get("crime id") or lower_map.get("crime_id")

# (defensive) parse month to ensure correct type if needed
if month_col in df_city.columns:
    df_city[month_col] = pd.to_datetime(df_city[month_col], errors="coerce")

# plotting the maps center point for proper visualisation
center_lat = float(df_city[lat_col].median())
center_lon = float(df_city[lon_col].median())

# building the map
m = folium.Map(location=[center_lat, center_lon], zoom_start=15, tiles="OpenStreetMap")

# adding controls to the map
MiniMap(toggle_display=True).add_to(m)
Fullscreen().add_to(m)
MeasureControl(primary_length_unit='meters', secondary_length_unit='miles').add_to(m)

# creating the heatmap of reported crime incidents
fg_heat = FeatureGroup(name="Crime Heatmap", show=True)
heat_data = df_city[[lat_col, lon_col]].values.tolist()
HeatMap(heat_data, radius=14, blur=10).add_to(fg_heat)
fg_heat.add_to(m)

# clustering the crime incident
fg_markers = FeatureGroup(name="Crime Markers (clustered)", show=True)
mc = MarkerCluster().add_to(fg_markers)

MAX_MARKERS = len(df_city)
pts = df_city.sample(n=min(MAX_MARKERS, len(df_city)), random_state=42) if len(df_city) > MAX_MARKERS else df_city

popup_cols = [c for c in [loc_col, month_col, crime_id_col] if c in df_city.columns]
for _, row in pts.iterrows():
    popup_text = []
    if loc_col in popup_cols:
        popup_text.append(f"<b>Location:</b> {row.get(loc_col, '')}")
    if month_col in popup_cols and pd.notna(row.get(month_col)):
        popup_text.append(f"<b>Month:</b> {str(row.get(month_col))[:10]}")
    if crime_id_col in popup_cols:
        popup_text.append(f"<b>Crime ID:</b> {row.get(crime_id_col, '')}")
    if not popup_text:
        popup_text.append(f"({row[lat_col]:.6f}, {row[lon_col]:.6f})")

    folium.Marker(
        [row[lat_col], row[lon_col]],
        popup=folium.Popup("<br>".join(popup_text), max_width=300)
    ).add_to(mc)

fg_markers.add_to(m)

# Layer control
LayerControl(collapsed=False).add_to(m)

# Saving the map to .html file
out_path = Path("C:/Users/rohit/OneDrive/Desktop/Uni/Dissertation/city_crime_folium_heat_plus_markers.html")
m.save(out_path)

print({"combined_map": str(out_path), "rows_plotted": len(df_city)})


{'combined_map': 'C:\\Users\\rohit\\OneDrive\\Desktop\\Uni\\Dissertation\\city_crime_folium_heat_plus_markers.html', 'rows_plotted': 16638}
